In [106]:
import csv
import pandas as pd
from svm_model.services.pre_process_text import remove_punctuations
from svm_model.services.pre_process_text import replace_special_chars
from svm_model.services.pre_process_text import remove_stop_words
from svm_model.services.pre_process_text import rejoin_text
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.neural_network import MLPClassifier
import pickle
from sklearn.externals import joblib

rating = {'1':'Very_Bad', '2':'Bad','3':'Neutral','4':'Good','5':'Very_Good'}


In [107]:
#pre-process review column 
df = pd.read_csv('dataset.csv',encoding='utf-8-sig', dtype = str)

In [ ]:
df['review'] = remove_punctuations(df['review'] )
df['review'] = df['review'].apply(replace_special_chars)
df['review'] = df['review'].apply(remove_stop_words)
df['review'] = df['review'].apply(rejoin_text)

In [102]:
#save pre-processing results to dataset file
df.to_csv('dataset.csv',encoding='utf-8-sig')

In [ ]:
#vectorize dataset column 'review'
#determine dataset input, and output
#split training data into training and testing set 80%, 20%
vectorizer = TfidfVectorizer()
corpus  = df['review'].values
x = vectorizer.fit_transform(corpus)
joblib.dump(vectorizer, "vectorizer.pkl")
y=df['rating']
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [108]:
vectorizer= joblib.load("vectorizer.pkl")

In [109]:
#create svm model
#fit training data 
lin_clf = svm.LinearSVC()
lin_clf.fit(x, y)
y_pred2 = lin_clf.predict(X_test)
print(classification_report(y_test, y_pred2))
print(accuracy_score(y_test, y_pred2)) 

              precision    recall  f1-score   support

           1       0.92      0.81      0.86      2846
           2       0.92      0.95      0.93      7817
           4       0.91      0.90      0.90      5249
           5       0.91      0.94      0.92      5228

    accuracy                           0.92     21140
   macro avg       0.91      0.90      0.91     21140
weighted avg       0.92      0.92      0.91     21140

0.9151844843897824


In [110]:
joblib.dump(lin_clf, "finalized_svm_model.pkl")

['finalized_svm_model.pkl']

In [113]:
clf = MLPClassifier(solver='lbfgs', alpha=0.001, hidden_layer_sizes=(5, 3), random_state=1)
clf.fit(x, y)
y_pred2=clf.predict(X_test)
print(classification_report(y_test, y_pred2))
print(accuracy_score(y_test, y_pred2)) 

c:\programdata\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


              precision    recall  f1-score   support

           1       0.81      0.76      0.78      2846
           2       0.89      0.89      0.89      7817
           4       0.83      0.83      0.83      5249
           5       0.87      0.90      0.89      5228

    accuracy                           0.86     21140
   macro avg       0.85      0.84      0.85     21140
weighted avg       0.86      0.86      0.86     21140

0.8595553453169347


In [114]:
joblib.dump(clf, "finalized_mlp_model.pkl")

['finalized_mlp_model.pkl']

In [115]:
mdl= joblib.load("finalized_mlp_model.pkl")
rev="جيد جدا، استمتع بالاقامة !"
rev = remove_punctuations(rev)
rev = replace_special_chars(rev)
rev = remove_stop_words(rev)
rev = rejoin_text(rev)
rev = vectorizer.transform([rev])
y_pred2 = mdl.predict(rev)
def predictMdl(rev,loaded_model):
    y_pred2 = mdl.predict(rev)
    print(str(type(mdl)).split(".")[-1][:-2],"\t",rating[y_pred2[0]])
    return y_pred2
pred=predictMdl(rev,mdl)


MLPClassifier 	 Good
